# Calculate vertical ensemble mean on regular pressure levels 
CESM2-LE ONLY

* **Description**: Reads in and creates seasonal and ensemble means and vertically interpolates
* **Input data**: CESM2-LE output in timeseries format from intake-esm
* **Output data**: Netcdf file with output
* **Creator**: Alice DuVivier
* **Date**: March 2022

In [1]:
import xarray as xr
import numpy as np
from datetime import timedelta
import glob

import pop_tools

import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.gridspec import GridSpec

import geocat.datafiles as gdf
import geocat.viz.util as gvutil
from geocat.viz import cmaps as gvcmaps
import geocat.comp as gcomp

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.stats import linregress,pearsonr, t

import dask
import intake
from distributed import Client
from ncar_jobqueue import NCARCluster

/glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
# spin up dask cluster

import dask

# Use dask jobqueue
from dask_jobqueue import PBSCluster

# Import a client
from dask.distributed import Client

# Setup your PBSCluster
cluster = PBSCluster(
    cores=36, # The number of cores you want
    memory='300 GB', # Amount of memory
    processes=9, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='$TMPDIR', # Use your local directory
    resource_spec='select=1:ncpus=36:mem=300GB', # Specify resources
    project='P93300665', # Input your project ID here
    walltime='06:00:00', # Amount of wall time
    interface='ib0', # Interface to use
)
# Scale up
cluster.scale(jobs=8)

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.63:41257,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Manually set variables

In [4]:
# list the variables to load
var_in_1 = 'T'
var_in_2 = 'TS'

## Load the CESM-LE data 

We will use [`intake-esm`](https://intake-esm.readthedocs.io/en/latest/), which is a data catalog tool.
It enables querying a database for the files we want, then loading those directly as an `xarray.Dataset`.

First step is to set the "collection" for the CESM-LE, which depends on a json file conforming to the [ESM Catalog Specification](https://github.com/NCAR/esm-collection-spec).

In [5]:
catalog_file = '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'

cat = intake.open_esm_datastore(catalog_file)

/glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/intake_esm/core.py:84: DtypeWarning: Columns (5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  self._df, self.catalog_file = _fetch_catalog(self.esmcol_data, esmcol_obj, csv_kwargs)


In [6]:
forcing = 'cmip6'  # do not want smbb data
expt = 'ssp370'
comp = 'atm'
freq = 'month_1'

subset_1 = cat.search(variable=var_in_1, forcing_variant=forcing, experiment=expt, component=comp, frequency=freq )
subset_2 = cat.search(variable=var_in_2, forcing_variant=forcing, experiment=expt, component=comp, frequency=freq )

In [7]:
subset_1.df.head()

,component,stream,case,member_id,variable,start_time,end_time,time_range,long_name,units,vertical_levels,frequency,path,experiment,forcing_variant,cesm_member_id,control_branch_year,cmip_experiment_id
0,atm,cam.h0,b.e21.BSSP370cmip6.f09_g17.LE2-1001.001,r1i1001p1f1,T,2015-01,2024-12,201501-202412,Temperature,K,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,ssp370,cmip6,1001.001,1001,CESM2_ssp370_r1i1001p1f1
1,atm,cam.h0,b.e21.BSSP370cmip6.f09_g17.LE2-1001.001,r1i1001p1f1,T,2025-01,2034-12,202501-203412,Temperature,K,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,ssp370,cmip6,1001.001,1001,CESM2_ssp370_r1i1001p1f1
2,atm,cam.h0,b.e21.BSSP370cmip6.f09_g17.LE2-1001.001,r1i1001p1f1,T,2035-01,2044-12,203501-204412,Temperature,K,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,ssp370,cmip6,1001.001,1001,CESM2_ssp370_r1i1001p1f1
3,atm,cam.h0,b.e21.BSSP370cmip6.f09_g17.LE2-1001.001,r1i1001p1f1,T,2045-01,2054-12,204501-205412,Temperature,K,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,ssp370,cmip6,1001.001,1001,CESM2_ssp370_r1i1001p1f1
4,atm,cam.h0,b.e21.BSSP370cmip6.f09_g17.LE2-1001.001,r1i1001p1f1,T,2055-01,2064-12,205501-206412,Temperature,K,1.0,month_1,/glade/campaign/cgd/cesm/CESM2-LE/timeseries/a...,ssp370,cmip6,1001.001,1001,CESM2_ssp370_r1i1001p1f1


In [8]:
# check that we only have cmip6, not smbb, data
member_id = list(subset_1.df.experiment.unique())
print(member_id)

['ssp370']


In [9]:
%%time
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets_1 = subset_1.to_dataset_dict(cdf_kwargs={'chunks': {'time':50}, 'decode_times': True})
#    dsets_1 = subset_1.to_dataset_dict(cdf_kwargs={'chunks': {'time':240}, 'decode_times': True})
    
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets_2 = subset_2.to_dataset_dict(cdf_kwargs={'chunks': {'time':50}, 'decode_times': True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


CPU times: user 13.5 s, sys: 527 ms, total: 14 s
Wall time: 18.2 s


In [10]:
# load in the future datasets
futures_1 = []
for key in sorted(dsets_1.keys()):
    futures_1.append(dsets_1[key])
    print(key)

futures_2 = []
for key in sorted(dsets_2.keys()):
    futures_2.append(dsets_2[key])
    print(key)

atm.ssp370.cam.h0.cmip6.T
atm.ssp370.cam.h0.cmip6.TS


In [11]:
future_ds_1 = xr.concat(futures_1, dim='member_id')
future_ds_2 = xr.concat(futures_2, dim='member_id')

In [12]:
future_ds_1.time

<xarray.DataArray 'time' (time: 1032)>
array([cftime.DatetimeNoLeap(2015, 2, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2015, 3, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2015, 4, 1, 0, 0, 0, 0, has_year_zero=True), ...,
       cftime.DatetimeNoLeap(2100, 11, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2100, 12, 1, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2101, 1, 1, 0, 0, 0, 0, has_year_zero=True)],
      dtype=object)
Coordinates:
  * time     (time) object 2015-02-01 00:00:00 ... 2101-01-01 00:00:00
Attributes:
    long_name:  time
    bounds:     time_bnds

In [13]:
# Shift months by one to be center of time period.
# Take average of the time bounds to get middle of month
# will lose some attributes with time, so may need to put this back in later...
future_ds_1['time'] = future_ds_1.time_bnds.load().mean(dim='nbnd').sel(member_id='r1i1281p1f1')
future_ds_2['time'] = future_ds_2.time_bnds.load().mean(dim='nbnd').sel(member_id='r1i1281p1f1')

In [14]:
# get just NH slice
future_ds_1_masked = future_ds_1.isel(lat=slice(164,192))
future_ds_2_masked = future_ds_2.isel(lat=slice(164,192))

In [15]:
# grab variables of interest
T_le = future_ds_1_masked[var_in_1]
TS_le = future_ds_2_masked[var_in_2]

In [16]:
T_le.persist()
TS_le.persist()

,Array,Chunk
Bytes,1.55 GiB,1.54 MiB
Shape,"(50, 1032, 28, 288)","(1, 50, 28, 288)"
Count,1300 Tasks,1300 Chunks
Type,float32,numpy.ndarray


## Calculate pressure on regular levels

### Get surface pressure

In [17]:
# We will need surface pressure and reference pressure for interpolation

# set a surface reference pressure [Pa]
p0 = 100000 

# set variable name
var_in_3 = 'PS'

In [18]:
# use same input parameters as above
#forcing = 'cmip6'  # do not want smbb data
#expt = 'ssp370'
#comp = 'atm'
#freq = 'month_1'

subset_3 = cat.search(variable=var_in_3, forcing_variant=forcing, experiment=expt, component=comp, frequency=freq )

In [19]:
%%time
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets_3 = subset_3.to_dataset_dict(cdf_kwargs={'chunks': {'time':240}, 'decode_times': True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


CPU times: user 8.84 s, sys: 174 ms, total: 9.01 s
Wall time: 9.24 s


In [20]:
# load in the future datasets
futures_3 = []
for key in sorted(dsets_3.keys()):
    futures_3.append(dsets_3[key])
    print(key)

atm.ssp370.cam.h0.cmip6.PS


In [21]:
future_ds_3 = xr.concat(futures_3, dim='member_id')

In [22]:
# Shift months by one to be center of time period.
# Take average of the time bounds to get middle of month
# will lose some attributes with time, so may need to put this back in later...
future_ds_3['time'] = future_ds_3.time_bnds.load().mean(dim='nbnd').sel(member_id='r1i1281p1f1')

In [23]:
# get just NH slice
future_ds_3_masked = future_ds_3.isel(lat=slice(164,192))

In [24]:
# grab variables of interest
PS_le = future_ds_3_masked[var_in_3]

### Load the vertical parameters

In [25]:
# grab parameters for interpolation
hyam_le = future_ds_1_masked["hyam"]
hybm_le = future_ds_1_masked["hybm"]

### Interpolate to new pressure levels from hybrid levels

In [26]:
# Specify output pressure levels
new_levels = np.array([925, 850, 700, 500])  # in millibars
new_levels = new_levels * 100  # convert to Pascals

In [27]:
%%time
T_le_interp = gcomp.interp_hybrid_to_pressure(T_le,
                                               PS_le,
                                               hyam_le, hybm_le, p0 = p0,
                                               new_levels=new_levels,
                                               method='linear')

CPU times: user 27.4 ms, sys: 0 ns, total: 27.4 ms
Wall time: 27.4 ms


In [28]:
T_le_interp.persist()

,Array,Chunk
Bytes,6.20 GiB,6.15 MiB
Shape,"(50, 1032, 4, 28, 288)","(1, 50, 4, 28, 288)"
Count,1300 Tasks,1300 Chunks
Type,float32,numpy.ndarray


## Now calculate temperature gradients that are relevant (based on wind speed aloft trends)

- TS with T925, T850, T700

In [29]:
Tgrad = T_le_interp - TS_le

In [30]:
Tgrad.persist()

,Array,Chunk
Bytes,6.20 GiB,6.15 MiB
Shape,"(50, 1032, 4, 28, 288)","(1, 50, 4, 28, 288)"
Count,1300 Tasks,1300 Chunks
Type,float32,numpy.ndarray


## Calculate Seasonal Means

In [31]:
season_names = ['OND','JFM', 'AMJ', 'JAS']

In [32]:
# find total years
xarr_le = Tgrad.coords['time.year'][(Tgrad.coords['time.month']==1)]

In [33]:
Tgrad

,Array,Chunk
Bytes,6.20 GiB,6.15 MiB
Shape,"(50, 1032, 4, 28, 288)","(1, 50, 4, 28, 288)"
Count,26861 Tasks,1300 Chunks
Type,float32,numpy.ndarray


In [34]:
# Loop through seasons - le

# make numpy array to fill and specify dimensions we want
seas_array_le_1 = np.zeros([len(season_names),len(xarr_le),len(Tgrad.member_id),len(Tgrad.plev),len(Tgrad.lat),len(Tgrad.lon)])
seas_array_le_2 = np.zeros([len(season_names),len(xarr_le),len(Tgrad.member_id),len(Tgrad.plev),len(Tgrad.lat),len(Tgrad.lon)])
seas_array_le_3 = np.zeros([len(season_names),len(xarr_le),len(Tgrad.member_id),len(Tgrad.lat),len(Tgrad.lon)])

for s_count, ss in enumerate(season_names):
#for s_count, ss in enumerate(season_names[0:1]):
    print(ss)
    ### TGRAD
    # get temporary array of just these month by season
    if ss == 'JFM':
        temp1 = Tgrad.isel(time=Tgrad.time.dt.month.isin([1,2,3]))
    if ss == 'AMJ':
        temp1 = Tgrad.isel(time=Tgrad.time.dt.month.isin([4,5,6]))
    if ss == 'JAS':
        temp1 = Tgrad.isel(time=Tgrad.time.dt.month.isin([7,8,9]))
    if ss == 'OND':
        temp1 = Tgrad.isel(time=Tgrad.time.dt.month.isin([10,11,12]))
    # now loop through years to get the seasonal average by year for each ensemble member
    for y_count, yy in enumerate(xarr_le):
    #for y_count, yy in enumerate(xarr_le[0:1]):
        # select only the indexes for this year
        temp1a = temp1.isel(time=temp1.time.dt.year.isin([yy])).mean(dim='time')
        seas_array_le_1[s_count,y_count,:,:,:,:] = temp1a
        
    ### TPLEV
    # get temporary array of just these month by season
    if ss == 'JFM':
        temp1 = T_le_interp.isel(time=T_le_interp.time.dt.month.isin([1,2,3]))
    if ss == 'AMJ':
        temp1 = T_le_interp.isel(time=T_le_interp.time.dt.month.isin([4,5,6]))
    if ss == 'JAS':
        temp1 = T_le_interp.isel(time=T_le_interp.time.dt.month.isin([7,8,9]))
    if ss == 'OND':
        temp1 = T_le_interp.isel(time=T_le_interp.time.dt.month.isin([10,11,12]))
    # now loop through years to get the seasonal average by year for each ensemble member
    for y_count, yy in enumerate(xarr_le):
    #for y_count, yy in enumerate(xarr_le[0:1]):
        # select only the indexes for this year
        temp1a = temp1.isel(time=temp1.time.dt.year.isin([yy])).mean(dim='time')
        seas_array_le_2[s_count,y_count,:,:,:,:] = temp1a   

    ### TS
    # get temporary array of just these month by season
    if ss == 'JFM':
        temp1 = TS_le.isel(time=TS_le.time.dt.month.isin([1,2,3]))
    if ss == 'AMJ':
        temp1 = TS_le.isel(time=TS_le.time.dt.month.isin([4,5,6]))
    if ss == 'JAS':
        temp1 = TS_le.isel(time=TS_le.time.dt.month.isin([7,8,9]))
    if ss == 'OND':
        temp1 = TS_le.isel(time=TS_le.time.dt.month.isin([10,11,12]))
    # now loop through years to get the seasonal average by year for each ensemble member
    for y_count, yy in enumerate(xarr_le):
    #for y_count, yy in enumerate(xarr_le[0:1]):
        # select only the indexes for this year
        temp1a = temp1.isel(time=temp1.time.dt.year.isin([yy])).mean(dim='time')
        seas_array_le_3[s_count,y_count,:,:,:] = temp1a   
        

OND
JFM
AMJ
JAS


In [35]:
print(seas_array_le_1.shape)
print(seas_array_le_2.shape)
print(seas_array_le_3.shape)

(4, 86, 50, 4, 28, 288)
(4, 86, 50, 4, 28, 288)
(4, 86, 50, 28, 288)


In [36]:
# convert the numpy array to a xarray for easier plotting
TGRAD_seas_le = xr.DataArray(seas_array_le_1,dims=('season','time','member_id','plev','lat','lon'))
T_seas_le = xr.DataArray(seas_array_le_2,dims=('season','time','member_id','plev','lat','lon'))
TS_seas_le = xr.DataArray(seas_array_le_3,dims=('season','time','member_id','lat','lon'))

In [37]:
# set coordinate arrays
TGRAD_seas_le['season'] = season_names
TGRAD_seas_le['time'] = xarr_le
TGRAD_seas_le['member_id'] = Tgrad['member_id']
TGRAD_seas_le['plev'] = Tgrad['plev'].values
TGRAD_seas_le['lat'] = Tgrad['lat'].values
TGRAD_seas_le['lon'] = Tgrad['lon'].values

T_seas_le['season'] = season_names
T_seas_le['time'] = xarr_le
T_seas_le['member_id'] = Tgrad['member_id']
T_seas_le['plev'] = Tgrad['plev'].values
T_seas_le['lat'] = Tgrad['lat'].values
T_seas_le['lon'] = Tgrad['lon'].values

TS_seas_le['season'] = season_names
TS_seas_le['time'] = xarr_le
TS_seas_le['member_id'] = Tgrad['member_id']
TS_seas_le['lat'] = Tgrad['lat'].values
TS_seas_le['lon'] = Tgrad['lon'].values


## Write out files

In [38]:
# quick and dirty way to save a file!

# save rufmod expt, rename the variable so it makes sense
#fout = 'rufmod_vertical_seas_ens_mean_WS'
#
#WS_seas_ens_mean_rufmod.to_dataset(name='vert_ws').to_netcdf(fout+'.nc')

### TGRAD

In [39]:
#set info to write out
out_tag = 'TGRAD'
units = 'K'
longname = 'temperature gradient (Taloft - Tsfc)'

fout = 'CESM2-LE_seas_'+out_tag

In [40]:
ds_to_save = TGRAD_seas_le

In [41]:
# check how big this will be to write out in GB
ds_to_save.nbytes/(1024**3)

4.13360595703125

In [42]:
# assign some attributes
refdata = {'Author': 'Alice DuVivier', 'units':units, 'longname':longname}

ds_to_save.attrs = refdata

In [43]:
# check data
ds_to_save


<xarray.DataArray (season: 4, time: 86, member_id: 50, plev: 4, lat: 28, lon: 288)>
array([[[[[[-6.77497244e+00, -6.98870707e+00, -7.29158831e+00, ...,
            -6.67165136e+00, -6.62774849e+00, -6.64881706e+00],
           [-6.64512205e+00, -6.84713888e+00, -7.09777880e+00, ...,
            -6.38741446e+00, -6.42524672e+00, -6.49750662e+00],
           [-6.56565857e+00, -6.74338102e+00, -6.95525360e+00, ...,
            -6.14450645e+00, -6.27494812e+00, -6.42284155e+00],
           ...,
           [ 1.04154503e+00,  1.03085220e+00,  1.02349246e+00, ...,
             1.10870397e+00,  1.08855557e+00,  1.06160283e+00],
           [ 5.19589603e-01,  5.16512454e-01,  5.13722777e-01, ...,
             5.35482109e-01,  5.28091133e-01,  5.23041904e-01],
           [ 1.13063335e-01,  1.11985050e-01,  1.11008488e-01, ...,
             1.16979755e-01,  1.15833007e-01,  1.14248432e-01]],

          [[-1.12214622e+01, -1.14303169e+01, -1.17545271e+01, ...,
            -1.12144575e+01, -1.11120958e+01, -1.11206694e+01],
           [-1.10317068e+01, -1.12415123e+01, -1.15306921e+01, ...,
            -1.08197441e+01, -1.08316545e+01, -1.08932829e+01],
           [-1.08997297e+01, -1.10895004e+01, -1.13436928e+01, ...,
            -1.04253044e+01, -1.05681238e+01, -1.07395277e+01],
...
           [-4.08417130e+00, -4.08541822e+00, -4.08711004e+00, ...,
            -4.07312822e+00, -4.07714033e+00, -4.08070421e+00],
           [-4.24821615e+00, -4.24772310e+00, -4.24783945e+00, ...,
            -4.24645185e+00, -4.24655581e+00, -4.24764872e+00],
           [-4.40998220e+00, -4.41005468e+00, -4.41008425e+00, ...,
            -4.40973663e+00, -4.40980959e+00, -4.40991116e+00]],

          [[-2.88971653e+01, -2.91996326e+01, -2.94030762e+01, ...,
            -2.78183041e+01, -2.81798763e+01, -2.85437145e+01],
           [-2.87578220e+01, -2.90288620e+01, -2.92324371e+01, ...,
            -2.77021580e+01, -2.80933609e+01, -2.84459972e+01],
           [-2.85902233e+01, -2.88140106e+01, -2.89894485e+01, ...,
            -2.76245136e+01, -2.79924469e+01, -2.83165913e+01],
           ...,
           [-1.89863987e+01, -1.89886475e+01, -1.89910946e+01, ...,
            -1.89846096e+01, -1.89845428e+01, -1.89851990e+01],
           [-1.91428890e+01, -1.91449757e+01, -1.91471996e+01, ...,
            -1.91402359e+01, -1.91409588e+01, -1.91417923e+01],
           [-1.93031139e+01, -1.93031750e+01, -1.93032513e+01, ...,
            -1.93028831e+01, -1.93029652e+01, -1.93030415e+01]]]]]])
Coordinates:
  * season     (season) <U3 'OND' 'JFM' 'AMJ' 'JAS'
  * time       (time) int64 2015 2016 2017 2018 2019 ... 2097 2098 2099 2100
  * member_id  (member_id) <U12 'r10i1181p1f1' 'r10i1231p1f1' ... 'r9i1301p1f1'
  * plev       (plev) int64 92500 85000 70000 50000
  * lat        (lat) float64 64.55 65.5 66.44 67.38 ... 87.17 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    Author:    Alice DuVivier
    units:     K
    longname:  temperature gradient (Taloft - Tsfc)

In [44]:
ds_to_save.to_netcdf(fout+'.nc')  # how to save file

### T @ PLEVS

In [45]:
#set info to write out
out_tag = 'T'
units = 'K'
longname = 'temperature aloft at given plevels'

fout = 'CESM2-LE_seas_'+out_tag

In [46]:
ds_to_save = T_seas_le

In [47]:
# check how big this will be to write out in GB
ds_to_save.nbytes/(1024**3)

4.13360595703125

In [48]:
# assign some attributes
refdata = {'Author': 'Alice DuVivier', 'units':units, 'longname':longname}

ds_to_save.attrs = refdata

In [49]:
# check data
ds_to_save


<xarray.DataArray (season: 4, time: 86, member_id: 50, plev: 4, lat: 28, lon: 288)>
array([[[[[[275.64187622, 275.64529419, 275.65762329, ...,
            275.6055603 , 275.6300354 , 275.6421814 ],
           [275.25863647, 275.29476929, 275.35641479, ...,
            275.04855347, 275.14273071, 275.21774292],
           [274.82281494, 274.91799927, 275.00582886, ...,
            274.43118286, 274.57717896, 274.70858765],
           ...,
           [261.5715332 , 261.59207153, 261.61291504, ...,
            261.52651978, 261.5440979 , 261.55383301],
           [261.27548218, 261.28436279, 261.29373169, ...,
            261.25015259, 261.25863647, 261.26672363],
           [260.89376831, 260.89376831, 260.89376831, ...,
            260.89376831, 260.89404297, 260.89376831]],

          [[271.19540405, 271.20370483, 271.19473267, ...,
            271.06274414, 271.14572144, 271.17028809],
           [270.87203979, 270.90039062, 270.92349243, ...,
            270.61621094, 270.73632812, 270.82192993],
           [270.48873901, 270.5718689 , 270.61737061, ...,
            270.15039062, 270.28399658, 270.39190674],
...
           [269.60647583, 269.60812378, 269.60934448, ...,
            269.61050415, 269.60870361, 269.60739136],
           [269.45639038, 269.45797729, 269.45901489, ...,
            269.45495605, 269.4559021 , 269.4559021 ],
           [269.30795288, 269.30795288, 269.3079834 , ...,
            269.30795288, 269.3079834 , 269.30795288]],

          [[256.18847656, 256.19863892, 256.22463989, ...,
            256.18063354, 256.15835571, 256.17095947],
           [255.9987793 , 256.0305481 , 256.06204224, ...,
            255.94573975, 255.93421936, 255.96165466],
           [255.83467102, 255.89988708, 255.9462738 , ...,
            255.72694397, 255.74195862, 255.78633118],
           ...,
           [254.70428467, 254.70489502, 254.7053833 , ...,
            254.6990509 , 254.7013092 , 254.70288086],
           [254.56175232, 254.56072998, 254.55969238, ...,
            254.56117249, 254.56147766, 254.56175232],
           [254.4148407 , 254.4148407 , 254.4148407 , ...,
            254.41479492, 254.4148407 , 254.4148407 ]]]]]])
Coordinates:
  * season     (season) <U3 'OND' 'JFM' 'AMJ' 'JAS'
  * time       (time) int64 2015 2016 2017 2018 2019 ... 2097 2098 2099 2100
  * member_id  (member_id) <U12 'r10i1181p1f1' 'r10i1231p1f1' ... 'r9i1301p1f1'
  * plev       (plev) int64 92500 85000 70000 50000
  * lat        (lat) float64 64.55 65.5 66.44 67.38 ... 87.17 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    Author:    Alice DuVivier
    units:     K
    longname:  temperature aloft at given plevels

In [50]:
ds_to_save.to_netcdf(fout+'.nc')  # how to save file

### TSFC

In [51]:
#set info to write out
out_tag = 'TS'
units = 'K'
longname = 'surface temperature'

fout = 'CESM2-LE_seas_'+out_tag

In [52]:
ds_to_save = TS_seas_le

In [53]:
# check how big this will be to write out in GB
ds_to_save.nbytes/(1024**3)

1.0334014892578125

In [54]:
# assign some attributes
refdata = {'Author': 'Alice DuVivier', 'units':units, 'longname':longname}

ds_to_save.attrs = refdata

In [55]:
# check data
ds_to_save


<xarray.DataArray (season: 4, time: 86, member_id: 50, lat: 28, lon: 288)>
array([[[[[282.41687012, 282.6340332 , 282.94924927, ...,
           282.27719116, 282.25778198, 282.29098511],
          [281.90377808, 282.14187622, 282.45419312, ...,
           281.4359436 , 281.56799316, 281.71524048],
          [281.38845825, 281.66137695, 281.96109009, ...,
           280.57568359, 280.85211182, 281.13143921],
          ...,
          [260.52999878, 260.56124878, 260.58944702, ...,
           260.41781616, 260.45553589, 260.49221802],
          [260.75588989, 260.76785278, 260.7800293 , ...,
           260.71466064, 260.73052979, 260.74368286],
          [260.78067017, 260.7817688 , 260.78274536, ...,
           260.77676392, 260.77819824, 260.7795105 ]],

         [[282.32385254, 282.52780151, 282.83670044, ...,
           282.29483032, 282.22268677, 282.21987915],
          [281.63473511, 281.8664856 , 282.20626831, ...,
           281.36334229, 281.40206909, 281.4836731 ],
          [280.90274048, 281.2149353 , 281.58520508, ...,
           280.28161621, 280.45465088, 280.665802  ],
...
          [273.63919067, 273.64538574, 273.65133667, ...,
           273.62380981, 273.62878418, 273.63354492],
          [273.57885742, 273.58078003, 273.58279419, ...,
           273.57241821, 273.57461548, 273.57681274],
          [273.54269409, 273.5428772 , 273.54302979, ...,
           273.54202271, 273.54226685, 273.54248047]],

         [[285.08566284, 285.39828491, 285.62771606, ...,
           283.99893188, 284.33822632, 284.71466064],
          [284.7565918 , 285.05941772, 285.29446411, ...,
           283.64785767, 284.02758789, 284.40768433],
          [284.42489624, 284.71389771, 284.93569946, ...,
           283.35144043, 283.73440552, 284.10293579],
          ...,
          [273.69064331, 273.69354248, 273.69644165, ...,
           273.68365479, 273.68582153, 273.68811035],
          [273.70462036, 273.70571899, 273.70690918, ...,
           273.70141602, 273.7024231 , 273.70352173],
          [273.71792603, 273.71801758, 273.7180481 , ...,
           273.71768188, 273.71777344, 273.71786499]]]]])
Coordinates:
  * season     (season) <U3 'OND' 'JFM' 'AMJ' 'JAS'
  * time       (time) int64 2015 2016 2017 2018 2019 ... 2097 2098 2099 2100
  * member_id  (member_id) <U12 'r10i1181p1f1' 'r10i1231p1f1' ... 'r9i1301p1f1'
  * lat        (lat) float64 64.55 65.5 66.44 67.38 ... 87.17 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    Author:    Alice DuVivier
    units:     K
    longname:  surface temperature

In [56]:
ds_to_save.to_netcdf(fout+'.nc')  # how to save file